### Script to output the TAVG variable from GHCN-Daily.
### NH stations are chosen that have data between 1979 and 2014 are chosen.


In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from math import nan
import sys
import matplotlib.pyplot as plt

### Specify output path

In [12]:
pathout="/project/cas/islas/python_savs/GHCND/"

### Specify start year and end year for analysis.  Generate time axis for output and remove Feb 29th

In [2]:
ystart=1979 ; yend=2014
nyears=yend-ystart+1

# setting up output calendar dates
timeout = pd.date_range(start=str(ystart)+'-01-01',end=str(yend)+'-12-31')
# remove Feb 29th
timeout = timeout[~((timeout.month == 2) & (timeout.day == 29))]

Reading in the station file (not really needed - the inventory is more useful)

In [3]:
#statfile="/project/mojave/observations/GHCN-Daily/ghcnd-stations.txt"
#namestr=[0,11] ; latstr=[12,20] ; lonstr=[21,30]
#f = open(statfile,"r")
#name=[] ; lon=[] ; lat=[]
#for line in f:
#    lon.append(line[lonstr[0]:lonstr[1]])
#    lat.append(line[latstr[0]:latstr[1]])
#    name.append(line[namestr[0]:namestr[1]])
#f.close()

### Reading in the inventory 

In [4]:
invent = "/project/mojave/observations/GHCN-Daily/ghcnd-inventory.txt"
namestr=[0,11] ; latstr=[12,20] ; lonstr=[21,30]
varstr=[31,35] ; startstr=[36,40] ; endstr=[41,45] 
f = open(invent,"r")
name=[] ; lon=[] ; lat=[] ; start=[] ; end=[]
for line in f:
    var=line[varstr[0]:varstr[1]]
    if (var=='TAVG'):
        name.append(line[namestr[0]:namestr[1]]) # station name
        lat.append(line[latstr[0]:latstr[1]]) # station latitude
        lon.append(line[lonstr[0]:lonstr[1]]) # station longitude
        start.append(line[startstr[0]:startstr[1]]) # start year of station data
        end.append(line[endstr[0]:endstr[1]]) # end year of station data
        
    

In [5]:
dictstat=[{'name':name, 'lat':lat, 'lon':lon, 'start':start, 'end':end} for name, lat, lon, start, end in zip (name, lat, lon, start, end)]

### Figure out the stations to use.  Here based on latitude and start and end year

In [6]:
statuse=[]
lonstation=[]
latstation=[]
for key in dictstat:
    lonflt = float(key['lon'])
    latflt = float(key['lat'])
    name = key['name']
    start = float(key['start'])
    end = float(key['end'])
    
    if ( (latflt >=0.1) and (start <= ystart) and (end >= yend) ):
        statuse.append(name)
        lonstation.append(lonflt)
        latstation.append(latflt)

Generate dates for a non-leap year year

### Specifying start day for each month and the number of years in each month

In [8]:
daymonstart=[0,31,59,90,120,151,181,212,243,273,304,334]
ndaysmon=[31,28,31,30,31,30,31,31,30,31,30,31]

### Grab all the data

In [ ]:
t2m = np.empty([len(statuse), nyears*365])
t2m[:,:] = nan
stationname=[]
stationlon=[]
stationlat=[]
yearstr=[11,15]
monstr=[15,17]
varstr=[17,21]
datastr=[21,269]



countstations=0
for istat in range(0, len(statuse),1):
    print( str(istat)+':'+statuse[istat])
    file="/project/mojave/observations/GHCN-Daily/ghcnd_all/"+str(statuse[istat])+'.dly'
    f= open(file,"r")
    stationdat = np.zeros([nyears*365])*nan # initialize data array for this station with nans
    for line in f:
        var=line[varstr[0]:varstr[1]]

        if (var == 'TAVG'): # pick out TAVG
            year=line[yearstr[0]:yearstr[1]]
            if ((float(year) >= ystart) and (float(year) <= yend)): # pick out data between ystart and yend
                mon=line[monstr[0]:monstr[1]] # the month of data
                data=line[datastr[0]:datastr[1]] # the data
                days = [data[i*8:i*8+8] for i in np.arange(0,31,1)] # splitting up the data into those of individual days
                datestrings = [mon+'-'+str(i+1).zfill(2) for i in np.arange(0,31,1)]
                mflag = [days[i][5] for i in np.arange(0,31,1)] # getting the mflag
                qflag = [days[i][6] for i in np.arange(0,31,1)] # getting the qflag
                sflag = [days[i][7] for i in np.arange(0,31,1)] # getting the sflag
                values = [days[i][1:5] for i in np.arange(0,31,1)] # getting the actual TAVG values
                values_np = np.array(values) # converting to a numpy array
                values_np = values_np.astype(np.float)
                
                # checking for missing data with the -9999 or 9999 values - set them to nans
                values_np[(values_np.astype(int) < -9998) | (values_np.astype(int) > 9998)] = nan
                
                # removing any that fail the quality control flag
                values_np[ np.array(qflag) != ' '] = nan
                
                imon = np.int(mon)-1
                stationdat[(np.int(year)-ystart)*365 + daymonstart[imon]: (np.int(year)-ystart)*365 + daymonstart[imon] + ndaysmon[imon]] = values_np[0:ndaysmon[imon]]/10.
                
    numnans = len(stationdat[stationdat == nan])
    if (numnans < 0.8*nyears*365): # only using stations that have data for at least 80% of the days
        t2m[countstations,:] = stationdat
        stationname.append(statuse[istat])
        stationlon.append(lonstation[istat])
        stationlat.append(latstation[istat])
    
    countstations = countstations+1 
    
t2m = t2m[0:countstations,:]
stationname = stationname[0:countstations]
stationlon = stationlon[0:countstations]
stationlat = stationlat[0:countstations]

### Converting to xarray data arrays

In [11]:
t2m_xr = xr.DataArray(t2m, coords=[stationname, timeout], dims=['station','time'], name='t2m')
station = xr.DataArray(stationname, coords=[stationname], dims=['station'], name='station')
lon = xr.DataArray(stationlon, coords=[stationname], dims=['station'], name='lon')
lat = xr.DataArray(stationlat, coords=[stationname], dims=['station'], name='lat')

### Output the data

In [13]:
t2m_xr.to_netcdf(pathout+"T2m_GHCND_NH_1979_2014.nc")
station.to_netcdf(pathout+"T2m_GHCND_NH_1979_2014.nc", mode="a")
lon.to_netcdf(pathout+"T2m_GHCND_NH_1979_2014.nc", mode="a")
lat.to_netcdf(pathout+"T2m_GHCND_NH_1979_2014.nc", mode="a")